In [1]:
import numpy as np
import skimage.io as sio
import cv2

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image
import copy
from pyquaternion import Quaternion

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud

import os
import os.path as osp
import json
import tqdm

In [2]:
def translation2transform(vec):
    i = np.eye(4)
    i[:3, -1] = vec
    return i

In [3]:
scene_path = "../../shubham/nerf_llff_data/fern/"
os.listdir(scene_path)

['simplices.npy',
 'trimesh.png',
 'sparse',
 'mpis4',
 'images_8',
 'database.db',
 'images',
 'images_4',
 'poses_bounds.npy']

In [4]:
from datasets import dataset_dict

/home2/sdokania/.local/lib/python3.8/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


In [5]:
print(scene_path)

../../shubham/nerf_llff_data/fern/


In [6]:
dataset_dict

{'blender': datasets.blender.BlenderDataset,
 'llff': datasets.llff.LLFFDataset,
 'nusc': datasets.nusc.NusDataset}

In [7]:
dataset = dataset_dict['llff'](scene_path)

In [8]:
dataset.poses[0][:3, :3].T

array([[ 0.99569469,  0.02503342,  0.0892492 ],
       [-0.02079598,  0.99862618, -0.04809664],
       [-0.09033061,  0.04603354,  0.99484737]])

In [9]:
dataset.all_rays.shape[0]

3619728

In [10]:
3619728/(504*378)

19.0

In [11]:
dataset.all_rgbs.shape

torch.Size([3619728, 3])

In [12]:
dataset.points.shape

(1, 3, 10091)

In [13]:
n_vals = 504*378
which_cam = 0
rays_o, rays_d = dataset.all_rays[which_cam * n_vals:n_vals*(which_cam+1), :3], dataset.all_rays[which_cam*n_vals:n_vals*(which_cam+1), 3:6]
near, far = dataset.all_rays[which_cam * n_vals:n_vals*(which_cam+1), 6:7], dataset.all_rays[which_cam*n_vals:n_vals*(which_cam+1), 7:8]

start = (rays_o + near * rays_d).numpy()
end = (rays_o + far * rays_d).numpy()

In [14]:
print(start[0], end[0])

[-0.8724618  0.5583312 -1.0913798] [-2.9662883  2.130121  -5.2885127]


In [15]:
import plotly.graph_objects as go


In [16]:
end[::500].shape

(382, 3)

In [17]:
# Ks, poses = read_meta("../sus_data/scaled_scene/")

# poses = np.array(poses)
# poses.shape

In [18]:
print(start[0], end[0])

[-0.8724618  0.5583312 -1.0913798] [-2.9662883  2.130121  -5.2885127]


In [19]:
poses = np.array(dataset.poses)
print(poses.shape)

(20, 3, 4)


In [20]:
# am, aM = pc.points.min(1), pc.points.max(1)

# print(am.shape, aM.shape)

In [22]:
fig = go.Figure()

skip = 3


fig.add_trace(
    go.Scatter3d(
        x=dataset.points[0, 0][::2],
        y=dataset.points[0, 1][::2],
        z=dataset.points[0, 2][::2],
        mode='markers',
        name='scene',
        marker=dict(size=0.75)
    )
)


fig.add_trace(
    go.Scatter3d(
        x=poses[:, 0, 3],
        y=poses[:, 1, 3],
        z=poses[:, 2, 3],
        mode='markers',
        name='cameras',
        marker=dict(size=2)
    )
)

# fig.add_trace(
#     go.Scatter3d(
#         x=end[::504, 0],
#         y=end[::504, 1],
#         z=end[::504, 2],
#         mode='markers',
#         name='ends',
#         marker=dict(size=2)
#     )
# )


xlines = []
ylines = []
zlines = []
for i in [0, 503, -504, -1]:
    xlines += [start[i, 0], end[i, 0], None]
    ylines += [start[i, 1], end[i, 1], None]
    zlines += [start[i, 2], end[i, 2], None]
    
for p in [[0, 503], [503, -1], [-504, -1], [-504, 0]]:
    xlines += [start[p[0], 0], start[p[1], 0], None]
    ylines += [start[p[0], 1], start[p[1], 1], None]
    zlines += [start[p[0], 2], start[p[1], 2], None]
    xlines += [end[p[0], 0], end[p[1], 0], None]
    ylines += [end[p[0], 1], end[p[1], 1], None]
    zlines += [end[p[0], 2], end[p[1], 2], None]

fig.add_trace(
    go.Scatter3d(
        x=xlines,
        y=ylines,
        z=zlines,
        mode='lines',
        name='frustum',
        marker=dict(size=1)
    )
)


fig.update_layout(
    # scene = dict(
    #     xaxis = dict(nticks=4, range=[300,500],),
    #                  yaxis = dict(nticks=4, range=[1100,1300],),
    #                  zaxis = dict(nticks=4, range=[-1,300],),),
    width=700,
    scene_aspectmode='data',
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()

In [ ]:
rays_o.mean(0)

tensor([ 6.9347e+02,  3.2900e+03, -1.0000e+00])

In [22]:
dataset.all_rays.shape[0]/(800*450)

13.0

In [77]:
print(start[0], end[0])
print(start[100], end[100])

[4.1237518e+02 1.1807368e+03 8.7553257e-01] [ 418.38623   1185.4009      -1.5970472]
[4.1221692e+02 1.1809008e+03 8.4288698e-01] [ 417.5948   1186.2207     -1.760275]


In [40]:
from datasets.ray_utils import *

In [41]:
drns = get_ray_directions(450, 800, Ks[0])

In [23]:
print(sio.imread("../sus_data/even_smaller_scene/images/CAM_FRONT_RIGHT_007.jpg").shape)
# print(os.listdir("../sus_data/even_smaller_scene/images")[:10])

(225, 400, 3)


In [24]:
op = dataset_val[0]

In [28]:
val_item = dataset_val[0]
rays_o, rays_d = val_item['rays'][:, :3], val_item['rays'][:, 3:6]
near, far = val_item['rays'][:, 6:7], val_item['rays'][:, 7:8]


In [29]:
rays_o.shape, rays_d.shape, near.shape, far.shape

(torch.Size([90000]),
 torch.Size([90000, 3]),
 torch.Size([90000, 1]),
 torch.Size([90000, 1]))

In [ ]:

start = (rays_o + near * rays_d).numpy()
end = (rays_o + far * rays_d).numpy()